# Paso 1. Limpieza de Datos

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
from sqlalchemy import create_engine, text
import psycopg2

In [2]:
#cargar archivo csv y leerlo
crediclub_csv = os.path.join("Data", "data.csv")
crediclub_df = pd.read_csv(crediclub_csv)
crediclub_df.head(5)

C:\Users\annar\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ID_Cliente,ID_Grupo,Ciclo,F_Inicio,Monto,Cuota,Plazo,Sucursal,Nacionalidad,F_Nacimiento,Sexo,Entidad_Nacimiento,Entidad_Residencia,CodigoPostal,IngresoMensual,Liquido,4/23/2023
0,32147.0,91.0,40.0,7/4/2022,79000.0,5490.48,16.0,70562.0,MEXICANA ...,2/26/1966,F,19,19.0,66056,90000.0,0.0,58
1,32143.0,91.0,40.0,7/4/2022,34000.0,2362.99,16.0,70562.0,MEXICANA ...,4/6/1966,F,32,19.0,66073,52000.0,0.0,58
2,32154.0,91.0,40.0,7/4/2022,70000.0,4864.98,16.0,70562.0,MEXICANA ...,3/4/1976,F,19,19.0,66056,70000.0,0.0,48
3,32151.0,91.0,40.0,7/4/2022,73000.0,5073.48,16.0,70562.0,MEXICANA ...,3/4/1976,F,19,19.0,66056,70000.0,0.0,48
4,32159.0,91.0,40.0,7/4/2022,16000.0,1112.00,16.0,70562.0,MEXICANA ...,2/11/1968,F,19,19.0,66056,30000.0,0.0,56


In [3]:
#revisar si hay valores nulos
valores_nulos = crediclub_df.isnull().sum()
valores_nulos

ID_Cliente            1814
ID_Grupo              1814
Ciclo                 1814
F_Inicio              1814
Monto                 1814
Cuota                 1814
Plazo                 1814
Sucursal              1814
Nacionalidad          1814
F_Nacimiento          1814
Sexo                  1814
Entidad_Nacimiento    1814
Entidad_Residencia    1814
CodigoPostal          1814
IngresoMensual        1814
Liquido               1814
4/23/2023             1814
dtype: int64

In [4]:
crediclub_df = crediclub_df.dropna(subset=['ID_Cliente'])

In [5]:
#revisar si hay valores nulos
valores_nulos = crediclub_df.isnull().sum()
valores_nulos

ID_Cliente            0
ID_Grupo              0
Ciclo                 0
F_Inicio              0
Monto                 0
Cuota                 0
Plazo                 0
Sucursal              0
Nacionalidad          0
F_Nacimiento          0
Sexo                  0
Entidad_Nacimiento    0
Entidad_Residencia    0
CodigoPostal          0
IngresoMensual        0
Liquido               0
4/23/2023             0
dtype: int64

In [6]:
crediclub_df.dtypes

ID_Cliente            float64
ID_Grupo              float64
Ciclo                 float64
F_Inicio               object
Monto                 float64
Cuota                 float64
Plazo                 float64
Sucursal              float64
Nacionalidad           object
F_Nacimiento           object
Sexo                   object
Entidad_Nacimiento     object
Entidad_Residencia    float64
CodigoPostal           object
IngresoMensual        float64
Liquido               float64
4/23/2023              object
dtype: object

In [7]:
#cambiamos datos a integers
crediclub_df['ID_Cliente'] = crediclub_df['ID_Cliente'].astype(int)
crediclub_df['ID_Grupo'] = crediclub_df['ID_Grupo'].astype(int)
crediclub_df['Ciclo'] = crediclub_df['Ciclo'].astype(int)
crediclub_df['Plazo'] = crediclub_df['Plazo'].astype(int)
crediclub_df['Sucursal'] = crediclub_df['Sucursal'].astype(int)
crediclub_df['Entidad_Residencia'] = crediclub_df['Entidad_Residencia'].astype(int)
crediclub_df['Liquido'] = crediclub_df['Liquido'].astype(int)

#cambiamos datos a fecha
crediclub_df['F_Inicio'] = pd.to_datetime(crediclub_df['F_Inicio'])

#cambiamos datos a strings
crediclub_df['Nacionalidad'] = crediclub_df['Nacionalidad'].astype(str)
crediclub_df['Sexo'] = crediclub_df['Sexo'].astype(str)

#en'Entidad_Nacimiento' hay errores como espacios vacios y hay que reemplazarlos por 0
crediclub_df['Entidad_Nacimiento'] = pd.to_numeric(crediclub_df['Entidad_Nacimiento'], errors='coerce')
crediclub_df['Entidad_Nacimiento'] = crediclub_df['Entidad_Nacimiento'].fillna(0)
crediclub_df['Entidad_Nacimiento'] = crediclub_df['Entidad_Nacimiento'].astype(int)

#en'CodigoPostal'hay errores como espacios vacios y hay que reemplazarlos por 0
crediclub_df['CodigoPostal'] = crediclub_df['CodigoPostal'].str.strip()
crediclub_df['CodigoPostal'] = crediclub_df['CodigoPostal'].replace('', np.nan)
crediclub_df['CodigoPostal'] = crediclub_df['CodigoPostal'].fillna(0)
crediclub_df['CodigoPostal'] = crediclub_df['CodigoPostal'].astype(int)

#en'4/23/2023'hay errores como espacios vacios y hay que reemplazarlos por 0
crediclub_df['4/23/2023'] = pd.to_numeric(crediclub_df['4/23/2023'], errors='coerce')
crediclub_df['4/23/2023'] = crediclub_df['4/23/2023'].fillna(0) 
crediclub_df['4/23/2023'] = crediclub_df['4/23/2023'].astype(int)

#en''F_Nacimiento'hay errores como espacios vacios y hay que quitarlos
crediclub_df['F_Nacimiento'] = pd.to_datetime(crediclub_df['F_Nacimiento'], errors='coerce')
crediclub_df.dropna(subset=['F_Nacimiento'], inplace=True)
crediclub_df['F_Nacimiento'] = pd.to_datetime(crediclub_df['F_Nacimiento'])

In [8]:
crediclub_df.dtypes

ID_Cliente                     int32
ID_Grupo                       int32
Ciclo                          int32
F_Inicio              datetime64[ns]
Monto                        float64
Cuota                        float64
Plazo                          int32
Sucursal                       int32
Nacionalidad                  object
F_Nacimiento          datetime64[ns]
Sexo                          object
Entidad_Nacimiento             int32
Entidad_Residencia             int32
CodigoPostal                   int32
IngresoMensual               float64
Liquido                        int32
4/23/2023                      int32
dtype: object

In [9]:
crediclub_df.head(5)

,ID_Cliente,ID_Grupo,Ciclo,F_Inicio,Monto,Cuota,Plazo,Sucursal,Nacionalidad,F_Nacimiento,Sexo,Entidad_Nacimiento,Entidad_Residencia,CodigoPostal,IngresoMensual,Liquido,4/23/2023
0,32147,91,40,2022-07-04,79000.0,5490.48,16,70562,MEXICANA ...,1966-02-26,F,19,19,66056,90000.0,0,58
1,32143,91,40,2022-07-04,34000.0,2362.99,16,70562,MEXICANA ...,1966-04-06,F,32,19,66073,52000.0,0,58
2,32154,91,40,2022-07-04,70000.0,4864.98,16,70562,MEXICANA ...,1976-03-04,F,19,19,66056,70000.0,0,48
3,32151,91,40,2022-07-04,73000.0,5073.48,16,70562,MEXICANA ...,1976-03-04,F,19,19,66056,70000.0,0,48
4,32159,91,40,2022-07-04,16000.0,1112.00,16,70562,MEXICANA ...,1968-02-11,F,19,19,66056,30000.0,0,56


In [10]:
crediclub_df.rename(columns = {'4/23/2023':'Edad_4_23_2023'}, inplace = True)
crediclub_df

,ID_Cliente,ID_Grupo,Ciclo,F_Inicio,Monto,Cuota,Plazo,Sucursal,Nacionalidad,F_Nacimiento,Sexo,Entidad_Nacimiento,Entidad_Residencia,CodigoPostal,IngresoMensual,Liquido,Edad_4_23_2023
0,32147,91,40,2022-07-04,79000.0,5490.48,16,70562,MEXICANA ...,1966-02-26,F,19,19,66056,90000.0,0,58
1,32143,91,40,2022-07-04,34000.0,2362.99,16,70562,MEXICANA ...,1966-04-06,F,32,19,66073,52000.0,0,58
2,32154,91,40,2022-07-04,70000.0,4864.98,16,70562,MEXICANA ...,1976-03-04,F,19,19,66056,70000.0,0,48
3,32151,91,40,2022-07-04,73000.0,5073.48,16,70562,MEXICANA ...,1976-03-04,F,19,19,66056,70000.0,0,48
4,32159,91,40,2022-07-04,16000.0,1112.00,16,70562,MEXICANA ...,1968-02-11,F,19,19,66056,30000.0,0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54062,3731304,268973,1,2022-07-29,15000.0,1110.01,16,70738,1,1957-06-30,F,27,27,86500,89000.0,1,67
54063,3731331,268973,1,2022-07-29,19000.0,1406.01,16,70738,1,1983-01-21,F,27,27,86599,89000.0,1,41
54064,3731372,268973,1,2022-07-29,25000.0,1850.02,16,70738,1,1982-11-23,F,27,27,86500,40000.0,1,41
54065,3731669,268973,1,2022-07-29,17000.0,1258.01,16,70738,1,2001-06-03,F,27,27,86500,31000.0,1,22


In [ ]:
#crediclub_df['Deudor_Pagador'] = np.where(crediclub_df['Liquido'] == 0, 'Deudor', 'Pagador')

In [ ]:
#crediclub_df.to_csv('crediclub_limpio.csv')

# Paso 2: Crear la tabla de destino en redshift

--- Crear tabla entidades-----
CREATE TABLE entidades (
id_entidad INT NOT NULL,
entidad VARCHAR NOT NULL,
PRIMARY KEY (id_entidad),
UNIQUE (id_entidad));

--- Crear tabla deudores-----
CREATE TABLE deudores (
id_deudor INT NOT NULL,
estatus VARCHAR NOT NULL,
PRIMARY KEY (id_deudor ),
UNIQUE (id_deudor));

--- Crear tabla destino-----
CREATE TABLE credi_club (
ID_Cliente INT NOT NULL, 
ID_Grupo	INT NOT NULL, 
Ciclo	INT NOT NULL, 
F_Inicio	DATE not null,
Monto	FLOAT not null,
Cuota	FLOAT not null,
Plazo	INT NOT NULL, 
Sucursal	INT NOT NULL, 
Nacionalidad	VARCHAR NOT NULL, 
F_Nacimiento	DATE not null,
Sexo	VARCHAR NOT NULL, 
Entidad_Nacimiento	INT NOT NULL, 
Entidad_Residencia	INT NOT NULL, 
CodigoPostal	INT NOT NULL, 
IngresoMensual	FLOAT not null,
Liquido	INT NOT NULL, 
Edad_4_23_2023	INT NOT NULL, 
FOREIGN KEY (Entidad_Nacimiento) REFERENCES entidades(id_entidad),
FOREIGN KEY (Entidad_Residencia) REFERENCES entidades(id_entidad),
FOREIGN KEY (Liquido) REFERENCES deudores (id_deudor));

# Paso 3: establecer conexion a Redshift¶

In [12]:
from config import redshift_pwd
from config import mysql_pwd
from config import user
from config import host
from config import data_base

In [15]:
try:
    conn = psycopg2.connect(
        host= host,
        dbname=data_base,
        user=user,
        password=redshift_pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


# Paso 4: Verificar que la tabla exista

In [16]:
cur = conn.cursor()
#ejecutar una query para verificar si existe la tabla que creamos
cur.execute("SELECT * FROM credi_club")
# traer los resultados
results = cur.fetchall()
results

[]

# Paso 5: Enviar la DF limpio a redshift

In [18]:
#llamamos a la funcion dentro de extras que trae valores, tambn lo hace sqlalquemy pero puede haber errores
#nos permite hacer un INSERT multiple con respecto a las columnas del Dataframe
from psycopg2.extras import execute_values

#establecer el cursor hacia la base de datos
cur = conn.cursor()

#ponemos el nombre de la tabla a la cual queremos insertar los datos
table_name = 'credi_club'

#ponemos las columnas 
columns = ['ID_Cliente', 'ID_Grupo', 'Ciclo', 'F_Inicio', 'Monto', 'Cuota', 
           'Plazo', 'Sucursal', 'Nacionalidad', 'F_Nacimiento', 'Sexo', 'Entidad_Nacimiento', 
           'Entidad_Residencia', 'CodigoPostal', 'IngresoMensual', 'Liquido', 'Edad_4_23_2023']

#sacamos los resultados que entran a una matriz en numpy y hace un ciclo para insertar los datos
#es una lista de tuplas
#cada tupla es una lista del data frame
values = [tuple(x) for x in crediclub_df.to_numpy()]

#aqui inserta los datos separados por comas y le estamos diciendo que es un string
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"

#comenzar la transaccion que necesita un inicio y una autorizacion 
#te pide el cursor, la query y los valores
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")


In [19]:
# SIEMPRE HAY QUE CERRAR EL CURSOR Y LA CONEXION!!!!
cur.close()
conn.close()